In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, CuDNNLSTM, LSTM

In [2]:
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [3]:
X_train = X_train/255
X_test = X_test/255

In [4]:
# standard LSTM
model = Sequential()
model.add(LSTM(units=128, input_shape=(X_train.shape[1:]), activation='relu', return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=128, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=10, activation='softmax'))

opt = tf.keras.optimizers.Adam(lr=1e-3, decay=1e-5)

model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [5]:
%%time
model.fit(X_train, y_train, batch_size=64, epochs=3, validation_data=(X_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 61s 1ms/step - loss: 0.7888 - acc: 0.7331 - val_loss: 0.1519 - val_acc: 0.9553
Epoch 2/3
60000/60000 [==============================] - 57s 954us/step - loss: 0.1871 - acc: 0.9483 - val_loss: 0.1688 - val_acc: 0.9588
Epoch 3/3
60000/60000 [==============================] - 56s 928us/step - loss: 0.1261 - acc: 0.9660 - val_loss: 0.0838 - val_acc: 0.9753
CPU times: user 5min 6s, sys: 18.1 s, total: 5min 24s
Wall time: 2min 55s


In [6]:
# cuDNN optmized LSTM
# it doesn't take the activation parameter
cudnn_model = Sequential()
cudnn_model.add(CuDNNLSTM(units=128, input_shape=(X_train.shape[1:]), return_sequences=True))
cudnn_model.add(Dropout(0.2))

cudnn_model.add(CuDNNLSTM(units=128))
cudnn_model.add(Dropout(0.2))

cudnn_model.add(Dense(units=32, activation='relu'))
cudnn_model.add(Dropout(0.2))

cudnn_model.add(Dense(units=10, activation='softmax'))

opt = tf.keras.optimizers.Adam(lr=1e-3, decay=1e-5)

cudnn_model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [7]:
%%time
cudnn_model.fit(X_train, y_train, batch_size=64, epochs=3, validation_data=(X_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 6s 101us/step - loss: 0.4711 - acc: 0.8524 - val_loss: 0.1409 - val_acc: 0.9578
Epoch 2/3
60000/60000 [==============================] - 5s 80us/step - loss: 0.1340 - acc: 0.9648 - val_loss: 0.0906 - val_acc: 0.9746
Epoch 3/3
60000/60000 [==============================] - 5s 80us/step - loss: 0.0957 - acc: 0.9751 - val_loss: 0.0745 - val_acc: 0.9788
CPU times: user 20.5 s, sys: 851 ms, total: 21.3 s
Wall time: 16.4 s
